In [1]:
# Import the data and label coordinates



# Create label maps from the coordinates



In [2]:
import matplotlib.pyplot as plt
import matplotlib as matplt
from torchvision import models, transforms
from PIL import Image

loader = transforms.Compose([
  #transforms.Resize(img_size),
  #transforms.CenterCrop(img_size),
  transforms.ToTensor(),
]) 

def load_image(filename, volatile=False):
    """
    Simple function to load and preprocess the image.

    1. Open the image.
    2. Scale/crop it and convert it to a float tensor.
    3. Convert it to a variable (all inputs to PyTorch models must be variables).
    4. Add another dimension to the start of the Tensor (b/c VGG expects a batch).
    5. Move the variable onto the GPU.
    """
    image = Image.open(filename).convert('RGB')
 
    '''
    #increase contrast
    hist, bins = np.histogram(image.flatten(),256,[0,256])
    cdf = hist.cumsum()
    cdf_normalized = cdf * hist.max()/ cdf.max()
    cdf_m = np.ma.masked_equal(cdf,0)
    cdf_m = (cdf_m - cdf_m.min())*255/(cdf_m.max()-cdf_m.min())
    cdf = np.ma.filled(cdf_m,0).astype('uint8')
    image = cdf[image]
    enhancer = ImageEnhance.Contrast(image)
    image = enhancer.enhance(3.0)
    '''
    
    image_tensor = loader(image).float()
    return image_tensor
    #image_var = Variable(image_tensor, volatile=volatile).unsqueeze(0)
    #return image_var.cuda

plt.imshow(load_image('data/2nirtrienvs/undistorted/210_left.jpg').numpy().transpose(1,2,0))

In [3]:
import torch
import torch.nn.functional as F
from torch import nn
CPU_DEBUG = False

# Setup the model
%env CUDA_LAUNCH_BLOCKING=1
device = torch.device("cpu" if CPU_DEBUG else "cuda")

class CoarseFireDetector(nn.Module):
    # Your code goes here
    def __init__(self):
        super(CoarseFireDetector, self).__init__()
                                                #input_channels, output_channels, kernel_size
        self.layers = nn.ModuleList([nn.Conv2d(3, 16, 5, stride=1, bias=True)])
        self.layers.append(nn.ReLU())
        self.layers.append(nn.Conv2d(16, 16, 5, stride=2, bias=True))
        self.layers.append(nn.ReLU())
        self.layers.append(nn.Conv2d(16, 8, 5, stride=1, bias=True))
        self.layers.append(nn.ReLU())
        self.layers.append(nn.Conv2d(8, 8, 3, stride=1, bias=True))
        self.layers.append(nn.ReLU())
        self.layers.append(nn.Conv2d(8, 1, 3, stride=1, bias=True))
        #self.layers.append(nn.Sigmoid())
    
    def forward(self, input):
        out = input 
        #print(input.dim())
        if input.dim() == 3:
            out = out.unsqueeze(0)
            for layer in self.layers:
                out = layer(out)
            return out.squeeze()
        else:
            for layer in self.layers:
                out = layer(out)
            return out 
    

env: CUDA_LAUNCH_BLOCKING=1


In [4]:
cf_detector = CoarseFireDetector().cuda()
out = cf_detector(load_image('data/2nirtrienvs/undistorted/210_left.jpg').cuda())
print(out.size())
plt.imshow(load_image('data/2nirtrienvs/undistorted/210_left.jpg').numpy().transpose(1,2,0))
plt.show()
plt.imshow(out.cpu().detach().numpy())
label_shape = out.cpu().detach().numpy().shape

RuntimeError: cuda runtime error (30) : unknown error at /opt/conda/conda-bld/pytorch_1544202130060/work/aten/src/THC/THCGeneral.cpp:51

In [ ]:
import os
import pickle
import pprint
import random

def show_torch_img(tensor):
    plt.imshow(tensor.detach().cpu().numpy().transpose(1,2,0))
    #plt.show()

pp = pprint.PrettyPrinter(indent=2)

# Training Dataset
data = "data/2nirtrienvs/undistorted"
metadp = "metadata.dat"
metadata = None
with open(os.path.join(data, metadp), 'rb') as metadf:
    print(os.path.join(data, metadp))
    metadata = pickle.load(metadf)
    #metadata = dict((k,metadata[k]) for k in range(478))
    #pp.pprint(metadata)

# load the input imgs
input_imgs = []
coords = []
for i, num in enumerate(metadata):
    #print(metadata[num])
    for side in 'left', 'right':
        print(metadata[num][side]['img_path'])
        if 'ml' in metadata[num][side]['img_path']:
            input_imgs.append(load_image(metadata[num][side]['img_path'][3:]))
        else:
            input_imgs.append(load_image(metadata[num][side]['img_path']))
        coords.append(metadata[num][side]['coords'])
    #print(coords[-2])
    #show_torch_img(input_imgs[-2])
    #show_torch_img(input_imgs[-1])

In [ ]:
import numpy as np
x, y = np.meshgrid(np.linspace(-1,1,9), np.linspace(-1,1,9))
d = np.sqrt(x*x+y*y)
sigma, mu = 0.4, 0.0
g = np.exp(-( (d-mu)**2 / ( 2.0 * sigma**2 ) ) )
print("2D Gaussian-like array:")
#print(g)
plt.imshow(g)
#print(g.shape)

labels = []
for num, img in enumerate(input_imgs):
    #print(num)
    # Create labels
    #print(metadata[num])
    #print(input_imgs[num])
    label = np.zeros(label_shape)
    coord = coords[num]
    _, input_imgs_w, input_imgs_h = input_imgs[num].numpy().shape
    #print(input_imgs_w, input_imgs_h)
    #print(label_shape)
    
    if coord == None:
        pass
    elif coord[0] == None or coord [1] == None:
        pass
    else:
    #print(coord)
        central = (int(coord[0]/input_imgs_w*label_shape[0]), 
                        int(coord[1]/input_imgs_h*label_shape[1]))
        #print(central)
        #print(central)
        for i in range(-4, 5):
            for j in range(-4, 5):
                lx = min(max(central[1]+i, 0),label_shape[0]-1)
                ly = min(max(central[0]+j, 0),label_shape[1]-1)
                label[lx, ly] += g[i+4, j+4]
        #show_torch_img(img)
    #plt.show()
    #plt.imshow(label)
    #plt.show()
    #show_torch_img
    labels.append(label)

In [ ]:
from torch import FloatTensor
from torch import optim

cf_detector = CoarseFireDetector().cuda()

optimizer = optim.Adam(cf_detector.parameters(), lr = 0.001) 

n_epochs = 3
for i in range(n_epochs):
    for j, (img, label) in enumerate(zip(input_imgs, labels)):
        if (j > 24 and j != 134 and j < 280) or (j > 390 and j < 424):
        #if j > 24 and j != 134 or j < 424:
            optimizer.zero_grad()
            out = cf_detector(img.cuda().unsqueeze(0))
            #print(out.size())
            weights = np.zeros_like(label)
            weights[label<0.001] = 1.2
            weights[label>=0.001] = 1.0
            loss = torch.mean(FloatTensor(weights).cuda()*(FloatTensor(label).cuda()-out)**2)
            #loss = torch.sum((FloatTensor(label).cuda()-out)**2)
            loss.backward()
            optimizer.step()
            #print(loss.item())


In [ ]:
errdists = []
trupos, falpos, truneg, falneg = 0, 0, 0, 0

for ind, (img, label) in enumerate(zip(input_imgs, labels)):
    if ind <= 24 or ind == 134 or ind >= 424:
        show_torch_img(img)
        '''
        outline = plt.Circle(coords[ind], 5, color='green', fill=False)
        ax = plt.gca()
        ax.add_patch(outline)
        plt.axis('scaled')
        '''
        plt.show()
        
        #show the ML prediction(probability of fire at all points on the image)
        ml_pred = cf_detector.cuda()(img.cuda()).cpu().detach().numpy()
        plt.imshow(ml_pred, cmap="seismic", vmin=0, vmax=0.001)
        
        #get coords of point most likely to be fire according to ML
        (predy, predx) = np.unravel_index(np.argmax(ml_pred), ml_pred.shape)
        print('ml prediction on current plot', (predy, predx))
        #NOTE: above line prints (y, x), NOT (x, y)
        plt.colorbar()
        plt.show()
        
        #show the human-labelled coordinates
        plt.imshow(label, cmap = "seismic", vmin=0, vmax=1.0)
        plt.colorbar()
        plt.show()
        
        #calculate real coords on new plot/grid
        x_factor = label.shape[0]/input_imgs[ind].shape[1]
        y_factor = label.shape[1]/input_imgs[ind].shape[2]
        if coords[ind] == None or None in coords[ind]:
            print('no fire')
            if ml_pred.max() > 0.04:
                falpos += 1
            elif ml_pred.max() <= 0.04:
                truneg += 1
        else:
            realy, realx = coords[ind][1]*y_factor, coords[ind][0]*x_factor
            #print('real coords on current plot', (realy, realx))
            #NOTE: above line prints (y, x), NOT (x, y)
            
            #confusion matrix
            if ml_pred.max() <= 0.04:
                falneg += 1
            elif ml_pred.max() > 0.04:
                trupos += 1
            
            #calculate straight-line distance btwn real coords and ML predictions
            #errdist = ( (x_factor*(realx-predx))**2 + (y_factor*(realy-predy))**2 )**0.5
            errdist = ( (predx/x_factor-coords[ind][0])**2 + (predy/y_factor-coords[ind][1])**2 )**0.5
            errdists.append(errdist)
            #print('distance btwn real and predicted coords on OG image =', errdist, 'pixels')
            #print('distance btwn real and predicted coords on OG image =', np.asarray(errdist)*1.4/1000, 'millimeters')

In [ ]:
#plot histogram of distances error
print(errdists)
#plt.hist(errdists, bins=list(range(1,7)))
plt.hist(errdists, bins=14)
plt.xlabel('Distance of Error(px)')
plt.ylabel('Count')
#plt.yticks(range(5))
plt.show()

print('mean distance =', sum(errdists)/len(errdists), 'px')

print('true positives', ':', trupos)
print('false positives', ':', falpos)
print('true negatives', ':', truneg)
print('false negatives', ':', falneg)